In [170]:
import pandas as pd
import time
import redis
from flask import current_app
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [171]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     #passwd="xpx",  # your password
                     db="xudatabase")        # name of the data base

cur = db.cursor()

In [172]:
# import article data from mysql db 
def import_data(table_name):
    #ds = pd.read_csv(data_source)
    ds = cur.execute('select * from ' +table_name)
    r = cur.fetchall()
    return pd.DataFrame(list(r),columns=['PMID', 'abstract'])
    

In [216]:
userpool = pd.read_csv("./data/testdata.csv")
userpool
userId =1

In [174]:
dailypool = import_data('articles')
dailypool.head()

,PMID,abstract
0,28274101,NADPH-P450 reductase (NPR) transfers electrons...
1,28274102,"Curcumin, a hydrophobic polyphenol derived fro..."
2,28274103,The single-chain variable fragment (scFv) agai...
3,28274104,The conformational change of cellular prion pr...
4,28274105,Alcoholic liver disease (ALD) is a leading cau...


In [3]:
def train(data_source):
    start = time.time()
    ds = import_data(data_source)
    info("Training data ingested in %s seconds." % (time.time() - start))

    start = time.time()
    self._train(ds)
    info("Engine trained in %s seconds." % (time.time() - start))    

In [4]:
# refactor the training logic into this function
def _train(**args):
    # place holder
    return 1;

In [175]:
ds = pd.concat([userpool, dailypool])
ds.reset_index(drop=True, inplace=True)
ds.head()

,PMID,abstract
0,27270041,Conserved DNA-damage responses (DDRs) sense ge...
1,19465921,Loss-of-function of caretaker genes characteri...
2,28258153,Pulmonary�tuberculosis�(TB) caused by�Mycobact...
3,25398087,Tuberculosis�(TB) is an infectious disease cau...
4,28274101,NADPH-P450 reductase (NPR) transfers electrons...


In [176]:
#clean up not ASCII chars
ds['abstract'] = ds["abstract"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
ds.head()

,PMID,abstract
0,27270041,Conserved DNA-damage responses (DDRs) sense ge...
1,19465921,Loss-of-function of caretaker genes characteri...
2,28258153,Pulmonary tuberculosis (TB) caused by Mycobact...
3,25398087,Tuberculosis (TB) is an infectious disease cau...
4,28274101,NADPH-P450 reductase (NPR) transfers electrons...


In [177]:
tf = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=0,
                             stop_words='english')

In [178]:
tfidf_matrix = tf.fit_transform(ds['abstract'])

In [179]:
tfidf_matrix

<832x13245 sparse matrix of type '<type 'numpy.float64'>'
	with 16571 stored elements in Compressed Sparse Row format>

In [180]:
cosine_similarities = linear_kernel(tfidf_matrix[:userpool.shape[0]], tfidf_matrix[:])
cosine_similarities

array([[ 1.        ,  0.08379392,  0.01243842, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08379392,  1.        ,  0.0050212 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01243842,  0.0050212 ,  1.        , ...,  0.        ,
         0.00803089,  0.        ],
       [ 0.01694175,  0.01779382,  0.17759771, ...,  0.00508897,
         0.01236981,  0.        ]])

In [181]:
similarities = pd.DataFrame(cosine_similarities)

In [182]:
similarities

,0,1,2,3,4,5,6,7,8,9,...,822,823,824,825,826,827,828,829,830,831
0,1.000000,0.083794,0.012438,0.016942,0.0,0.0,0.000000,0.000000,0.008321,0.000000,...,0.000000,0.0,0.0,0.0,0.009336,0.0,0.0,0.000000,0.000000,0.0
1,0.083794,1.000000,0.005021,0.017794,0.0,0.0,0.000000,0.007117,0.003506,0.021368,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,0.012438,0.005021,1.000000,0.177598,0.0,0.0,0.006828,0.000000,0.017010,0.000000,...,0.007471,0.0,0.0,0.0,0.008379,0.0,0.0,0.000000,0.008031,0.0
3,0.016942,0.017794,0.177598,1.000000,0.0,0.0,0.007011,0.010552,0.003938,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.005089,0.012370,0.0


In [183]:
prediction_matrix = pd.DataFrame(data=userpool.PMID, columns=['PMID'], index=userpool.index)
similarity_scores = prediction_matrix.copy()
similarity_scores

,PMID
0,27270041
1,19465921
2,28258153
3,25398087


In [184]:
top_n = 12
if (ds.shape[0] < top_n):
    top_n = ds.shape[0] + 1

for i in range(1,top_n-1):
    col_name = i
    prediction_matrix[col_name] = None
    similarity_scores[col_name] = -1
import datetime
now = datetime.datetime.now()

In [223]:
for idx in range(userpool.shape[0]):
    print "idx: %s" % idx
    similar_indices = cosine_similarities[idx].argsort()[:-top_n:-1]
    print "similar indices: %s" % similar_indices
    similar_items = [(cosine_similarities[idx][i], ds['PMID'][i]) for i in similar_indices]
    print "similar items: \n %s" % similar_items
    for i in range(1,top_n-1): 
           
        prediction_matrix.ix[idx, i] = similar_items[i][1] # insert into db directly 
        similarity_scores.ix[idx, i] = similar_items[i][0] # insert into a db table with date
        
    query = 'insert into recommend_articles values ("%s", %d, %d,\
        %d, %d, %d, %d, %d,\
        %d, %d, %d, %d, %d,\
        %.6f,%.6f,%.6f,%.6f,%.6f,\
        %.6f,%.6f,%.6f,%.6f,%.6f)'%(now, userId,userpool.ix[idx,'PMID'],\
                                                     similar_items[1][1],similar_items[2][1],\
                                                     similar_items[3][1],similar_items[4][1],similar_items[5][1],\
                                                     similar_items[6][1],similar_items[7][1],similar_items[8][1],\
                                                     similar_items[9][1],similar_items[10][1],similar_items[1][0],\
                                                     similar_items[2][0],\
                                                     similar_items[3][0],similar_items[4][0],similar_items[5][0],\
                                                     similar_items[6][0],similar_items[7][0],similar_items[8][0],\
                                                     similar_items[9][0],similar_items[10][0])
     
    cur.execute(query)
    db.commit()

idx: 0
similar indices: [  0   1  54 114 716 722 477 430 623 583 644]
similar items: 
 [(0.99999999999999933, 27270041), (0.083793924816965695, 19465921), (0.081937461746571905, 28274158), (0.072891501922273075, 28274224), (0.044891564175493434, 28274951), (0.041857678838202947, 28274958), (0.041100569768590646, 28274652), (0.036343454977238954, 28274595), (0.034456518980895613, 28274838), (0.033416943214084893, 28274794), (0.033103645640632812, 28274862)]
idx: 1
similar indices: [  1   0 446  53 283 716  89 565  54 159 639]
similar items: 
 [(1.0000000000000002, 19465921), (0.083793924816965695, 27270041), (0.044319576133340331, 28274615), (0.036427143370675, 28274157), (0.025591237986448155, 28274419), (0.024294734106412176, 28274951), (0.023544763988480065, 28274198), (0.02349890390335941, 28274768), (0.02340229811863526, 28274158), (0.022885901792000285, 28274274), (0.022311327257618148, 28274857)]
idx: 2
similar indices: [  2   3 808 116 468 807 777 622 646 326 664]
similar items:

In [221]:
prediction_matrix

,PMID,1,2,3,4,5,6,7,8,9,10
0,27270041,19465921,28274158,28274224,28274951,28274958,28274652,28274595,28274838,28274794,28274862
1,19465921,27270041,28274615,28274157,28274419,28274951,28274198,28274768,28274158,28274274,28274857
2,28258153,25398087,28275074,28274226,28274638,28275073,28275038,28274837,28274864,28274463,28274882
3,25398087,28258153,28275074,28274226,28274638,28274144,28274530,28274791,28274838,28274380,28274158


In [195]:
similarity_scores

,PMID,1,2,3,4,5,6,7,8,9,10
0,27270041,0.083794,0.081937,0.072892,0.044892,0.041858,0.041101,0.036343,0.034457,0.033417,0.033104
1,19465921,0.083794,0.044320,0.036427,0.025591,0.024295,0.023545,0.023499,0.023402,0.022886,0.022311
2,28258153,0.177598,0.128780,0.081987,0.044680,0.038990,0.027158,0.025657,0.024295,0.024143,0.024107
3,25398087,0.177598,0.101743,0.074897,0.062071,0.052757,0.045164,0.039715,0.030140,0.029770,0.029735


In [214]:
cur.execute('insert into last_pmid values ("%s", 27270041)'%now)
db.commit()
    

In [ ]:
db.close()